In [72]:
## Looks like Churchland lab has analyzed this data on cloud. Let's see if it has the outputs we want
import fastplotlib as fpl
import numpy as np
import os
from typing import *
import tifffile
from one.api import ONE
one = ONE()
assert not one.offline, 'ONE must be connect to Alyx for searching imaging sessions'
from wfield import *
import wfield.cli as wfield_cli
import pathlib
from pathlib import Path
%load_ext autoreload

In [71]:
"""
The two datasets we will process are: 
(1) d34a502f-bd06-471f-8334-df41f785e1d9 (CSK-im-011)
(2) 3158300f-e72c-42fc-ac6c-c981615fe00f (CSK-im-001)

"""

In [70]:
eid_used = '3158300f-e72c-42fc-ac6c-c981615fe00f'
one.list_datasets(eid_used)

In [69]:
collection = "raw_widefield_data"
raw_data_output = one.load_dataset(eid_used, 'imaging.frames.mov', collection=collection, download_only=True)

collection = "alf/widefield"

#Shape (2, height, width)
frame_average = one.load_dataset(eid_used, 'widefieldChannels.frameAverage.npy', collection=collection) 

#Shape (height, width, 200)
wf_U = one.load_dataset(eid_used, 'widefieldU.images.npy', collection=collection)

#Shape (200, 100000)
wf_SVT = one.load_dataset(eid_used, 'widefieldSVT.uncorrected.npy', collection=collection)[:, :100000]

#Shape (200, 10000)
wf_SVT_Hemo = one.load_dataset(eid_used, 'widefieldSVT.haemoCorrected.npy', collection=collection)[:, :100000]



# Run the widefield analysis on the downloaded data

In [6]:
raw_data_output = pathlib.Path("/data/lab/IBL_Alyx/zadorlab/Subjects/CSK-im-001/2021-01-19/001/raw_widefield_data/imaging.frames.mov")

In [68]:
# Run motion correction: 
wfield_cli._motion(raw_data_output.parent, nchannels=2)

In [67]:
wfield_cli._baseline(raw_data_output.parent, 30, nchannels=2)

# Here is where we define the mask

In [53]:
raw_data_output = pathlib.Path("/data/lab/IBL_Alyx/zadorlab/Subjects/CSK-im-001/2021-01-19/001/raw_widefield_data/imaging.frames.mov")
dat = load_stack(raw_data_output.parent, nchannels=2)
std_img_gcamp = get_std_mask(dat[:, 0], threshold=64)
std_img_blood = get_std_mask(dat[:, 1], threshold=62)

In [54]:
updated_mask = np.logical_or(std_img_gcamp.astype('bool'), std_img_blood.astype('bool')).astype("float32")

In [66]:
iw = fpl.ImageWidget(data = [np.asarray(dat[:, 0]), std_img_gcamp, std_img_blood, updated_mask],
                    figure_shape = (1, 4))
iw.cmap = "gray"
iw.show()

In [56]:
save_path = raw_data_output.parent / "manual_mask.npy"
np.save(str(save_path), std_img)